In [2]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/ibonreinoso/qiskit-hackathon-bilbao-19/master/DAX_PERFORMANCE_INDEX.csv"
data = pd.read_csv(url, sep=';')
data = data.drop(['wkn_500340'], axis = 1)
data = data.loc[:,['wkn_515100', 'wkn_575200']]

covariance_matrix = np.cov(data.values.T)

In [3]:
print(covariance_matrix)

[[ 67.38376849  97.4907718 ]
 [ 97.4907718  152.27294829]]


In [4]:
trace_normalize_matrix = covariance_matrix /np.matrix.trace(covariance_matrix)

print(trace_normalize_matrix)

[[0.30676853 0.44383242]
 [0.44383242 0.69323147]]


In [5]:
from scipy.linalg import expm

unitary = expm(2*1j*np.pi*trace_normalize_matrix)

print(unitary)

[[9.94996262e-01+0.03988286j 2.19878137e-16-0.09160674j]
 [2.24840803e-16-0.09160674j 9.94996262e-01-0.03988286j]]


In [7]:
from qiskit import *
qreg = QuantumRegister(3)
creg = ClassicalRegister(3)

qcircuit = QuantumCircuit(qreg, creg)


In [24]:
from qiskit.aqua.components.uncertainty_problems import UncertaintyProblem
from qiskit.aqua.algorithms.single_sample.amplitude_estimation.q_factory import QFactory


class PrincipalComponentAnalysisAFactory(UncertaintyProblem):
    """
    Circuit Factory representing the given covariance matrix and 
    iterates using a random vector which a given initial state.
    This matrix is used to initialize to construct Q and the 
    initial value of the random initial vector.
    """
    
    def __init__(self, random_vector, covariance_matrix):
        super().__init__(1)
        self._random_vector = random_vector
        self._covariance_matrix = covariance_matrix
        self.i_state = 0
        self._trace_normalize_matrix = covariance_matrix /np.matrix.trace(covariance_matrix)
        (self.th1, self.ph1, self.lam1) = qiskit.quantum_info.synthesis.two_qubit_decompose.euler_angles_1q(expm(2*1j*np.pi*self._trace_normalize_matrix))  
    
    def build(self, qc, q, q_ancillas=None):
        qc.initialize(self._random_vector, 2)
        qc.cu3(self.th1, self.ph1, self.lam1, 1, 2)



In [41]:
class PrincipalComponentAnalysisQFactory(QFactory):
    """
    Circuit Factory representing the operator Q.
    This implementation exploits the fact that powers of Q can be implemented efficiently by just multiplying the angle.
    (amplitude estimation only requires controlled powers of Q, thus, only this method is overridden.)
    """
    
    def __init__(self, qpca_expected_value):
        super().__init__(qpca_expected_value, i_objective=0)
    
    def build(self, qc, q, q_ancillas=None):
        i_state = self.a_factory.i_state
        trace_normalize_matrix = self.a_factory._trace_normalize_matrix
        
        (self.th1, self.ph1, self.lam1) = qiskit.quantum_info.synthesis.two_qubit_decompose.euler_angles_1q(expm(2*1j*np.pi*self._trace_normalize_matrix))  
        # Q is a rotation of angle 2*theta_p around the Y-axis
        qc.initialize(random_vector, 2)
    
    def build_controlled_power(self, qc, q, q_control, power, q_ancillas=None, use_basis_gates=True):
        i_state = self.a_factory.i_state

        qc.cu3(self.a_factory.th1, self.a_factory.ph1, self.a_factory.lam1, 1, 2)
        (th2, ph2, lam2) = qiskit.quantum_info.synthesis.two_qubit_decompose.euler_angles_1q(expm(2*1j*np.pi*self.a_factory._trace_normalize_matrix*2))
        qc.cu3(th2, ph2, lam2, 0, 2)

In [42]:
# construct factories for A and Q
qpca_a_factory = PrincipalComponentAnalysisAFactory([0,1], trace_normalize_matrix)
qpca_q_factory = PrincipalComponentAnalysisQFactory(qpca_a_factory)


In [43]:
from qiskit.aqua.algorithms import AmplitudeEstimation

# set number of evaluation qubits
m = 3

# construct amplitude estimation
# here, we override the standard construction of Q since we know a more efficient way
# (exploiting the fact that A and Q are just Y-rotations)
ae = AmplitudeEstimation(m, qpca_a_factory, q_factory=qpca_q_factory)

In [44]:
result = ae.run(quantum_instance=BasicAer.get_backend('statevector_simulator'))


In [47]:
ae._circuit.draw()


In [48]:
print(result)

{'statevector': array([[ 0.02277901+0.16184019j, -0.02382799-0.10282148j,
        -0.01964614-0.0179653j ,  0.02069512-0.04105341j,
         0.51418793-0.51664074j,  0.12510929+0.11025271j,
         0.33158112+0.00553146j,  0.01962056+0.53837716j,
         0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ]]), 'shots': 1, 'a_items': [(0.0, 0.026711129241929723), (0.1464466, 0.8215418169329101), (0.5, 0.0028223931313840155), (0.8535534, 0.13778463142913305), (1.0, 0.011140029264643105)], 'y_items': [(0, 0.026711129241929723), (1, 0.5313068796007325), (2, 0.0007087226304230802), (3, 0.10997663580290644), (4, 0.011140029264643105), (5, 0.027807995626226628), (6, 0.0021136705009609353), (7, 0.2902349373321775)], 'mapped_values': [0.0, 0.1464466, 0.5, 0.8535534, 1.0], 'values': [0.0, 0.1464466, 0.5, 0.8535534, 1.0], 'y_val